# Rearrange data

In [1]:
import os, sys
WORKING_DIR = os.getcwd()

#floyd
#DATA_HOME_DIR='/input'
#OUTPUT_DIR='/output'

#offline
DATA_HOME_DIR='/media/prit/Edu/Codes/ML/Fast.ai/courses/deeplearning1/nbs/data/dogscats'
OUTPUT_DIR='/media/prit/Edu/Codes/ML/Fast.ai/courses/deeplearning1/nbs/data/dogscats'

In [ ]:
%cd $DATA_HOME_DIR
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid

In [ ]:
%cd $DATA_HOME_DIR/train
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(2000): os.rename(shuf[i], DATA_HOME_DIR+'/valid/' + shuf[i])

In [ ]:
from shutil import copyfile
%cd $DATA_HOME_DIR/train
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(200): copyfile(shuf[i], DATA_HOME_DIR+'/sample/train/' + shuf[i])

%cd $DATA_HOME_DIR/valid
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(50): copyfile(shuf[i], DATA_HOME_DIR+'/sample/valid/' + shuf[i])

In [ ]:
#Divide cat/dog images into separate directories

%cd $DATA_HOME_DIR/sample/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/sample/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

# Create single 'unknown' class for test set
%cd $DATA_HOME_DIR/test1
%mv *.jpg unknown/

In [ ]:
%cd $WORKING_DIR

In [4]:
path = DATA_HOME_DIR + '/sample/'
test_path = DATA_HOME_DIR + '/test1/' #We use all the test data
results_path=OUTPUT_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'
model_path = OUTPUT_DIR+'/model/'
if not os.path.exists(model_path): os.mkdir(model_path)

OSError: [Errno 2] No such file or directory: '/media/prit/Edu/Codes/ML/Fast.ai/courses/deeplearning1/nbs/data/dogscats/model/'

# Training a better model

In [1]:
from theano.sandbox import cuda

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.


In [3]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

ModuleNotFoundError: No module named 'cPickle'

In [3]:
#path = DATA_HOME_DIR+'/'
path = DATA_HOME_DIR+'/sample/'
model_path = OUTPUT_DIR+'/model/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [ ]:
batch_size=64
no_of_epochs=3

In [4]:
model = vgg_ft(2)

In [5]:
layers = model.layers

In [6]:
last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Convolution2D][-1]

In [9]:
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)
# Dense layers - also known as fully connected or 'FC' layers
fc_layers = layers[last_conv_idx+1:]

Now we can use the exact same approach to creating features as we used when we created the linear model from the imagenet predictions in the last lesson - it's only the model that has changed. As you're seeing, there's a fairly small number of "recipes" that can get us a long way!

In [10]:
batches = get_batches(path+'train', shuffle=False, batch_size=batch_size)
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [11]:
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)

In [12]:
trn_features = conv_model.predict_generator(batches, batches.nb_sample)

In [13]:
save_array(model_path + 'train_convlayer_features.bc', trn_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In [14]:
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')

In [15]:
trn_features.shape

(160, 512, 14, 14)

In [30]:
# Ensure that we return to theano dimension ordering
K.set_image_dim_ordering('th')

## Adding data augmentation

Let's try adding a small amount of data augmentation, and see if we reduce overfitting as a result. The approach will be identical to the method we used to finetune the dense layers in lesson 2, except that we will use a generator with augmentation configured. Here's how we set up the generator, and create batches from it:

In [31]:
gen = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, 
                               height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True)

In [32]:
batches = get_batches(path+'train', gen, batch_size=batch_size)
# NB: We don't want to augment or shuffle the validation set
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


## Batch normalization

### About batch normalization

Batch normalization (*batchnorm*) is a way to ensure that activations don't become too high or too low at any point in the model. Adjusting activations so they are of similar scales is called *normalization*. Normalization is very helpful for fast training - if some activations are very high, they will saturate the model and create very large gradients, causing training to fail; if very low, they will cause training to proceed very slowly. Furthermore, large or small activations in one layer will tend to result in even larger or smaller activations in later layers, since the activations get multiplied repeatedly across the layers.

Prior to the development of batchnorm in 2015, only the inputs to a model could be effectively normalized - by simply subtracting their mean and dividing by their standard deviation. However, weights in intermediate layers could easily become poorly scaled, due to problems in weight initialization, or a high learning rate combined with random fluctuations in weights.

Batchnorm resolves this problem by normalizing each intermediate layer as well. The details of how it works are not terribly important (although I will outline them in a moment) - the important takeaway is that **all modern networks should use batchnorm, or something equivalent**. There are two reasons for this:
1. Adding batchnorm to a model can result in **10x or more improvements in training speed**
2. Because normalization greatly reduces the ability of a small number of outlying inputs to over-influence the training, it also tends to **reduce overfitting**.

As promised, here's a brief outline of how batchnorm works. As a first step, it normalizes intermediate layers in the same way as input layers can be normalized. But this on its own would not be enough, since the model would then just push the weights up or down indefinitely to try to undo this normalization. Therefore, batchnorm takes two additional steps:
1. Add two more trainable parameters to each layer - one to multiply all activations to set an arbitrary standard deviation, and one to add to all activations to set an arbitary mean
2. Incorporate both the normalization, and the learnt multiply/add parameters, into the gradient calculations during backprop.

This ensures that the weights don't tend to push very high or very low (since the normalization is included in the gradient calculations, so the updates are aware of the normalization). But it also ensures that if a layer does need to change the overall mean or standard deviation in order to match the output scale, it can do so.

### Adding batchnorm to the model

We can use nearly the same approach as before - but this time we'll add batchnorm layers (and dropout layers):

In [42]:
conv_layers[-1].output_shape[1:]

(512, 14, 14)

In [37]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(4096, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(1000, activation='softmax')
        ]

In [38]:
p=0.6

In [39]:
bn_model = Sequential(get_bn_layers(p))

In [40]:
def proc_wgts(layer, prev_p, new_p):
    scal = (1-prev_p)/(1-new_p)
    return [o*scal for o in layer.get_weights()]

In [41]:
for l in bn_model.layers: 
    if type(l)==Dense: l.set_weights(proc_wgts(l, 0.3, 0.6))

In [42]:
bn_model.pop()
for layer in bn_model.layers: layer.trainable=False

In [43]:
bn_model.add(Dense(2,activation='softmax'))

In [44]:
bn_model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

In [49]:
bn_model.fit(trn_features, trn_labels, nb_epoch=8, validation_data=(val_features, val_labels))

Train on 160 samples, validate on 40 samples
Epoch 1/8
160/160 [==============================] - 2s - loss: 0.1320 - acc: 0.9625 - val_loss: 0.1396 - val_acc: 0.9250
Epoch 2/8
160/160 [==============================] - 2s - loss: 0.1445 - acc: 0.9500 - val_loss: 0.1467 - val_acc: 0.9250
Epoch 3/8
160/160 [==============================] - 2s - loss: 0.2125 - acc: 0.9062 - val_loss: 0.1547 - val_acc: 0.9250

In [50]:
bn_model.save_weights(model_path+'bn.h5')

In [51]:
bn_model.load_weights(model_path+'bn.h5')

In [52]:
bn_layers = get_bn_layers(0.6)
bn_layers.pop()
bn_layers.append(Dense(2,activation='softmax'))

In [53]:
final_model = Sequential(conv_layers)
for layer in final_model.layers: layer.trainable = False
for layer in bn_layers: final_model.add(layer)

In [54]:
for l1,l2 in zip(bn_model.layers, bn_layers):
    l2.set_weights(l1.get_weights())

In [55]:
final_model.compile(optimizer=Adam(), 
                    loss='categorical_crossentropy', metrics=['accuracy'])
counter=0
learning_rate=0.01

In [ ]:
final_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/1
 96/160 [=================>............] - ETA: 55s - loss: 0.4128 - acc: 0.8229

In [60]:
counter+=1
final_model.save_weights(model_path + 'final{}.h5'.format(counter))

In [63]:
learning_rate/=10
final_model.optimizer.lr=learning_rate

In [1]:
batches, preds = vgg.test(test_path, batch_size = batch_size*2)

NameError: name 'vgg' is not defined

In [ ]:
filenames = batches.filenames
save_array(results_path + 'test_preds.dat', preds)
save_array(results_path + 'filenames.dat', filenames)